1. Install dan Import Library

In [17]:
# Cell 1: Install dan Import Library
!pip install openpyxl

import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Library berhasil diimport!")

✅ Library berhasil diimport!


2. Upload File Excel

In [18]:
# Cell 2: Upload File Excel
from google.colab import files
uploaded = files.upload()

# Load dataset
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name)

print("📊 Data berhasil dimuat!")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

Saving inspektorat-nominatif-surat-masuk-dari-juli-2022.xlsx to inspektorat-nominatif-surat-masuk-dari-juli-2022 (1).xlsx
📊 Data berhasil dimuat!
Shape: (162, 8)
Columns: ['Agenda', 'Surat', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Tanggal', 'Unnamed: 6', 'Pengolah']


3. Eksplorasi Data Awal

In [19]:
# Cek informasi dataset
print("Informasi dataset:")
print(df.info())

# Cek missing values
print("\nMissing values per kolom:")
print(df.isnull().sum())

# Cek duplikat
print(f"\nJumlah data duplikat: {df.duplicated().sum()}")

# Tampilkan statistik deskriptif
print("\nStatistik deskriptif:")
df.describe(include='all')

Informasi dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Agenda      161 non-null    float64
 1   Surat       162 non-null    object 
 2   Unnamed: 2  162 non-null    object 
 3   Unnamed: 3  162 non-null    object 
 4   Unnamed: 4  162 non-null    object 
 5   Tanggal     162 non-null    object 
 6   Unnamed: 6  162 non-null    object 
 7   Pengolah    161 non-null    object 
dtypes: float64(1), object(7)
memory usage: 10.3+ KB
None

Missing values per kolom:
Agenda        1
Surat         0
Unnamed: 2    0
Unnamed: 3    0
Unnamed: 4    0
Tanggal       0
Unnamed: 6    0
Pengolah      1
dtype: int64

Jumlah data duplikat: 0

Statistik deskriptif:


,Agenda,Surat,Unnamed: 2,Unnamed: 3,Unnamed: 4,Tanggal,Unnamed: 6,Pengolah
count,161.000000,162,162,162,162,162,162,161
unique,NaN,141,38,70,133,22,18,66
top,NaN,-,04/07/2022,SETDA,ABSENSI,05/07/2022,22/07/2022,IRBAN 2
freq,NaN,20,16,39,16,21,32,23
mean,217.490683,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,49.636569,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,134.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,174.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,219.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,259.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


4. Data Cleansing: Struktur Data

In [20]:
# Bersihkan nama kolom
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
print("Nama kolom setelah cleaning:", df.columns.tolist())

# Hapus baris yang seluruhnya kosong
df_clean = df.dropna(how='all')
print(f"Data setelah hapus baris kosong: {df_clean.shape}")

# Hapus baris header yang terduplikasi (jika ada)
df_clean = df_clean[df_clean['agenda'] != 'Agenda']

Nama kolom setelah cleaning: ['agenda', 'surat', 'unnamed:_2', 'unnamed:_3', 'unnamed:_4', 'tanggal', 'unnamed:_6', 'pengolah']
Data setelah hapus baris kosong: (162, 8)


5. Cleaning Kolom Tanggal

In [21]:
# Hapus kolom tanggal surat (kolom C) karena kurang relevan untuk analisis proses
print("Menghapus kolom tanggal surat...")
if 'tanggal' in df_clean.columns:
    df_clean = df_clean.drop('tanggal', axis=1)
    print("✅ Kolom 'tanggal' berhasil dihapus")
else:
    print("ℹ️ Kolom 'tanggal' tidak ditemukan")

# Fungsi cleaning tanggal (sama seperti sebelumnya)
def clean_date(date_str):
    """Fungsi untuk membersihkan format tanggal"""
    if pd.isna(date_str) or date_str == '' or date_str == '-':
        return np.nan

    # Convert to string
    date_str = str(date_str).strip()

    # Handle berbagai format tanggal
    try:
        # Format DD/MM/YYYY
        if '/' in date_str:
            return pd.to_datetime(date_str, dayfirst=True, errors='coerce')
        else:
            return pd.to_datetime(date_str, errors='coerce')
    except:
        return np.nan

# Cleaning TANGGAL TERIMA (kolom F)
if 'tanggal_terima' in df_clean.columns:
    df_clean['tanggal_terima_clean'] = df_clean['tanggal_terima'].apply(clean_date)
    print(f"✅ Tanggal Terima - Data valid: {df_clean['tanggal_terima_clean'].notna().sum()}")
else:
    # Cari alternatif nama kolom
    for col in ['unnamed:_5', 'unnamed:_6']:
        if col in df_clean.columns:
            df_clean['tanggal_terima_clean'] = df_clean[col].apply(clean_date)
            print(f"✅ Tanggal Terima (dari {col}) - Data valid: {df_clean['tanggal_terima_clean'].notna().sum()}")
            break

# Cleaning TANGGAL DISPOSISI (kolom G)
if 'disposisi' in df_clean.columns:
    df_clean['tanggal_disposisi_clean'] = df_clean['disposisi'].apply(clean_date)
    print(f"✅ Tanggal Disposisi - Data valid: {df_clean['tanggal_disposisi_clean'].notna().sum()}")
else:
    # Cari alternatif nama kolom
    for col in ['unnamed:_6', 'unnamed:_7']:
        if col in df_clean.columns and 'tanggal_terima_clean' not in df_clean.columns or col != 'tanggal_terima':
            df_clean['tanggal_disposisi_clean'] = df_clean[col].apply(clean_date)
            print(f"✅ Tanggal Disposisi (dari {col}) - Data valid: {df_clean['tanggal_disposisi_clean'].notna().sum()}")
            break

Menghapus kolom tanggal surat...
✅ Kolom 'tanggal' berhasil dihapus
✅ Tanggal Terima (dari unnamed:_6) - Data valid: 161
✅ Tanggal Disposisi (dari unnamed:_6) - Data valid: 161


6. Hitung Durasi Proses

In [22]:
print("\n" + "="*50)
print("MENGHITUNG DURASI PROSES SURAT")
print("="*50)

if 'tanggal_terima_clean' in df_clean.columns and 'tanggal_disposisi_clean' in df_clean.columns:
    df_clean['durasi_proses_hari'] = (df_clean['tanggal_disposisi_clean'] - df_clean['tanggal_terima_clean']).dt.days

    # Statistik durasi
    durasi_valid = df_clean['durasi_proses_hari'].notna()
    print(f"✅ Durasi proses berhasil dihitung")
    print(f"   - Rata-rata durasi: {df_clean[durasi_valid]['durasi_proses_hari'].mean():.1f} hari")
    print(f"   - Durasi tercepat: {df_clean[durasi_valid]['durasi_proses_hari'].min()} hari")
    print(f"   - Durasi terlama: {df_clean[durasi_valid]['durasi_proses_hari'].max()} hari")

    # Identifikasi data dengan masalah
    durasi_negatif = df_clean[df_clean['durasi_proses_hari'] < 0]
    print(f"   - Data dengan durasi negatif (perlu dicek): {len(durasi_negatif)}")
else:
    print("❌ Tidak bisa menghitung durasi - kolom tanggal tidak lengkap")



MENGHITUNG DURASI PROSES SURAT
✅ Durasi proses berhasil dihitung
   - Rata-rata durasi: 0.0 hari
   - Durasi tercepat: 0.0 hari
   - Durasi terlama: 0.0 hari
   - Data dengan durasi negatif (perlu dicek): 0


7. Cleaning Kolom Teks

In [23]:
print("\n" + "="*50)
print("CLEANING KOLOM TEKS")
print("="*50)

def clean_text(text):
    """Fungsi untuk membersihkan teks"""
    if pd.isna(text):
        return "Tidak Tersedia"

    text = str(text)
    text = re.sub(r'\s+', ' ', text)  # Ganti multiple spaces dengan single space
    text = text.strip()

    return text

# Apply cleaning ke kolom teks yang tersisa
text_columns = ['surat', 'dari', 'perihal', 'pengolah']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[f'{col}_clean'] = df_clean[col].apply(clean_text)
        print(f"✅ Kolom '{col}' berhasil dibersihkan")


CLEANING KOLOM TEKS
✅ Kolom 'surat' berhasil dibersihkan
✅ Kolom 'pengolah' berhasil dibersihkan


8. Standarisasi Format

In [24]:
print("\n" + "="*50)
print("STANDARDISASI FORMAT")
print("="*50)

# Standardisasi kolom 'dari'
def standardize_sender(sender):
    """Standardisasi nama pengirim"""
    if pd.isna(sender) or sender == 'Tidak Tersedia':
        return 'Tidak Tersedia'

    sender = str(sender).upper().strip()

    # Standardisasi singkatan umum
    replacements = {
        'DISKOMINFO': 'DINAS KOMUNIKASI DAN INFORMATIKA',
        'SETDA': 'SEKRETARIAT DAERAH',
        'BKD': 'BADAN KEPEGAWAIAN DAERAH',
        'DPRD': 'DEWAN PERWAKILAN RAKYAT DAERAH',
        'BKPSDM': 'BADAN KEPEGAWAIAN DAN PENGEMBANGAN SUMBER DAYA MANUSIA',
        'DISPARPORA': 'DINAS PARIWISATA PEMUDA DAN OLAHRAGA',
        'DISDIKBUD': 'DINAS PENDIDIKAN DAN KEBUDAYAAN'
    }

    for short, full in replacements.items():
        if short in sender:
            return full

    return sender

if 'dari_clean' in df_clean.columns:
    df_clean['dari_standard'] = df_clean['dari_clean'].apply(standardize_sender)
    print(f"✅ Standardisasi pengirim selesai")

# Standardisasi kolom 'pengolah'
def standardize_processor(processor):
    """Standardisasi nama pengolah"""
    if pd.isna(processor) or processor == 'Tidak Tersedia':
        return 'Tidak Tersedia'

    processor = str(processor).upper().strip()

    # Standardisasi unit kerja
    replacements = {
        'IRBAN': 'INSPEKTUR BANTUAN',
        'P.INSPEKTUR': 'INSPEKTUR',
        'KA UMUM': 'KEPALA BAGIAN UMUM',
        'KASUBBAG': 'KEPALA SUBBAGIAN',
        'MB.': 'MEMERIKSA BUKTI',
        'B.': 'BAGIAN'
    }

    for short, full in replacements.items():
        if short in processor:
            processor = processor.replace(short, full)

    return processor

if 'pengolah_clean' in df_clean.columns:
    df_clean['pengolah_standard'] = df_clean['pengolah_clean'].apply(standardize_processor)
    print(f"✅ Standardisasi pengolah selesai")


STANDARDISASI FORMAT
✅ Standardisasi pengolah selesai


9. Validasi dan Quality Check

In [25]:
# ===== LANGKAH 9: VALIDASI DAN QUALITY CHECK (lanjutan) =====
print("\n" + "="*50)
print("VALIDASI DATA CLEANING")
print("="*50)

print("📊 HASIL AKHIR DATA CLEANING:")
print(f"   - Total data: {len(df_clean)}")
print(f"   - Data dengan agenda valid: {df_clean['agenda'].notna().sum()}")

if 'tanggal_terima_clean' in df_clean.columns:
    print(f"   - Data dengan tanggal terima valid: {df_clean['tanggal_terima_clean'].notna().sum()}")

if 'tanggal_disposisi_clean' in df_clean.columns:
    print(f"   - Data dengan tanggal disposisi valid: {df_clean['tanggal_disposisi_clean'].notna().sum()}")

if 'durasi_proses_hari' in df_clean.columns:
    durasi_valid = df_clean['durasi_proses_hari'].notna()
    print(f"   - Data dengan durasi valid: {durasi_valid.sum()}")

    # Analisis kinerja
    surat_tuntas = df_clean['tanggal_disposisi_clean'].notna().sum()
    print(f"   - Surat sudah disposisi: {surat_tuntas} ({surat_tuntas/len(df_clean)*100:.1f}%)")



VALIDASI DATA CLEANING
📊 HASIL AKHIR DATA CLEANING:
   - Total data: 162
   - Data dengan agenda valid: 161
   - Data dengan tanggal terima valid: 161
   - Data dengan tanggal disposisi valid: 161
   - Data dengan durasi valid: 161
   - Surat sudah disposisi: 161 (99.4%)


10. Dataframe Final

In [26]:
print("\n" + "="*50)
print("MEMBUAT DATAFRAME FINAL")
print("="*50)

# Pilih hanya kolom yang sudah dibersihkan untuk disimpan
final_columns = ['agenda']

# Tambahkan kolom yang sudah dibersihkan
clean_columns_mapping = {
    'surat_clean': 'no_surat',
    'dari_standard': 'pengirim',
    'perihal_clean': 'perihal',
    'tanggal_terima_clean': 'tanggal_terima',
    'tanggal_disposisi_clean': 'tanggal_disposisi',
    'pengolah_standard': 'pengolah'
}

for clean_col, final_name in clean_columns_mapping.items():
    if clean_col in df_clean.columns:
        final_columns.append(clean_col)

# Tambahkan durasi jika ada
if 'durasi_proses_hari' in df_clean.columns:
    final_columns.append('durasi_proses_hari')

# Buat dataframe final
df_final = df_clean[final_columns]

# Rename kolom untuk hasil final
rename_mapping = {}
for col in df_final.columns:
    if col in clean_columns_mapping:
        rename_mapping[col] = clean_columns_mapping[col]
    elif col == 'durasi_proses_hari':
        rename_mapping[col] = 'durasi_hari'

df_final = df_final.rename(columns=rename_mapping)

print("✅ DataFrame final berhasil dibuat")
print(f"   - Kolom final: {df_final.columns.tolist()}")
print(f"   - Shape final: {df_final.shape}")

# Tampilkan sample data final
print("\n📋 SAMPLE DATA FINAL:")
df_final.head(10)


MEMBUAT DATAFRAME FINAL
✅ DataFrame final berhasil dibuat
   - Kolom final: ['agenda', 'no_surat', 'tanggal_terima', 'tanggal_disposisi', 'pengolah', 'durasi_hari']
   - Shape final: (162, 6)

📋 SAMPLE DATA FINAL:


,agenda,no_surat,tanggal_terima,tanggal_disposisi,pengolah,durasi_hari
0,NaN,No,NaT,NaT,Tidak Tersedia,NaN
1,304.0,028/141.1/VII/2022,2022-08-01,2022-08-01,INSPEKTUR BANTUAN 2,0.0
2,303.0,005/354.2/2022,2022-08-01,2022-08-01,INSPEKTUR,0.0
3,302.0,005/3.734.1.6,2022-08-01,2022-08-01,INSPEKTUR HADIR,0.0
4,301.0,-,2022-08-01,2022-08-01,INSPEKTUR HADIR,0.0
5,300.0,-,2022-08-01,2022-08-01,KEPALA BAGIAN UMUM,0.0
6,299.0,-,2022-08-01,2022-08-01,KEPALA BAGIAN UMUM,0.0
7,298.0,12/RT/VII/2022,2022-08-01,2022-08-01,KEPALA BAGIAN UMUM DAN MB ENDANG HARTI,0.0
8,297.0,-,2022-07-29,2022-07-29,INSPEKTUR...KEPALA BAGIAN UMUM KIRIM KE DISPER...,0.0
9,296.0,961/2584.7.2/VII/2022,2022-07-28,2022-07-28,INSPEKTUR BANTUAN 3,0.0


In [27]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_final)

https://docs.google.com/spreadsheets/d/1TV1KZuCYH5rbtbkNQAJrepxCo5Etcfyi4Xzi6tvritE/edit#gid=0


11. Simpan Hasil

In [29]:
print("\n" + "="*50)
print("MENYIMPAN HASIL")
print("="*50)

import os

# Define the output directory and create it if it doesn't exist
output_dir = '/content/Data Mining'
os.makedirs(output_dir, exist_ok=True)

# Define the output path
output_path = os.path.join(output_dir, 'surat_masuk_cleaned.csv')

# Simpan ke file CSV
df_final.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ Data berhasil disimpan ke: {output_path}")


MENYIMPAN HASIL
✅ Data berhasil disimpan ke: /content/Data Mining/surat_masuk_cleaned.csv


12. Summary Akhir

In [30]:
print("\n" + "="*50)
print("SUMMARY DATA CLEANING")
print("="*50)

print(f"📈 PERBANDINGAN AWAL vs AKHIR:")
print(f"   - Data awal: {df.shape}")
print(f"   - Data akhir: {df_final.shape}")
print(f"   - Kolom yang dihapus: 'tanggal surat'")
print(f"   - Kolom baru: 'durasi_hari'")

if 'durasi_hari' in df_final.columns:
    durasi_valid = df_final['durasi_hari'].notna()
    print(f"\n⏱️  METRIK KINERJA:")
    print(f"   - Rata-rata durasi proses: {df_final[durasi_valid]['durasi_hari'].mean():.1f} hari")
    print(f"   - Persentase surat tuntas: {df_final['tanggal_disposisi'].notna().sum()/len(df_final)*100:.1f}%")

print(f"\n🎯 DATA SIAP UNTUK ANALISIS!")


SUMMARY DATA CLEANING
📈 PERBANDINGAN AWAL vs AKHIR:
   - Data awal: (162, 8)
   - Data akhir: (162, 6)
   - Kolom yang dihapus: 'tanggal surat'
   - Kolom baru: 'durasi_hari'

⏱️  METRIK KINERJA:
   - Rata-rata durasi proses: 0.0 hari
   - Persentase surat tuntas: 99.4%

🎯 DATA SIAP UNTUK ANALISIS!


In [31]:
# Cara untuk mengurutkan data secara Ascending

print("\n" + "="*50)
print("PENGURUTAN DATA FINAL")
print("="*50)

# Langsung urutkan dataframe final yang sudah ada
df_final = df_final.sort_values('agenda', ascending=True)

print("✅ Data berhasil diurutkan berdasarkan agenda (terkecil → terbesar)")
print(f"📊 Agenda: {df_final['agenda'].iloc[0]} (terkecil) → {df_final['agenda'].iloc[-1]} (terbesar)")

# Tampilkan hasil
df_final.head(10)


PENGURUTAN DATA FINAL
✅ Data berhasil diurutkan berdasarkan agenda (terkecil → terbesar)
📊 Agenda: 134.0 (terkecil) → nan (terbesar)


,agenda,no_surat,tanggal_terima,tanggal_disposisi,pengolah,durasi_hari
161,134.0,-,2022-07-01,2022-07-01,INSPEKTUR BANTUAN 4,0.0
160,135.0,310/80.12.08/VI/2022,2022-07-01,2022-07-01,INSPEKTUR BANTUAN 4,0.0
159,136.0,005/324.2/2022,2022-07-01,2022-07-01,BAGIANENDAH,0.0
158,137.0,005/3.212.13,2022-07-01,2022-07-01,BAGIANENDAH DAN INSPEKTUR BANTUAN 1,0.0
157,138.0,822.3/524.15/2022,2022-07-01,2022-07-01,INSPEKTUR BANTUAN 4,0.0
156,139.0,640/18/2022,2022-07-04,2022-07-04,INSPEKTUR BANTUAN 1,0.0
155,140.0,005/0010325,2022-07-04,2022-07-04,BAGIANENDAH DAN MB ENDANG SRI SUHARTI,0.0
154,141.0,-,2022-07-04,2022-07-04,INSPEKTUR BANTUAN 3,0.0
153,142.0,892/1421.22/2022,2022-07-04,2022-07-04,NANDIKA DAN WAHYU KRISTIYANTO,0.0
152,143.0,030/429.25.17,2022-07-04,2022-07-04,INSPEKTUR BANTUAN 2,0.0


In [32]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_final)

https://docs.google.com/spreadsheets/d/16vJLnzSNHey5OjEp3Uki-q8an6JKBB7UMmu-vTe2um4/edit#gid=0
